# Grey Clustering Analysis - Assessment of Natural Disaster Events in the Rio Grande do Sul Area

## 1. Disaster Data

In [2]:
import pandas as pd

In [3]:
BASE_FILE_PATH  = '../Data/Danos_Informados_{year}.xls'
AVAILABLE_YEARS = range(2013, 2024)
disasters_data  = pd.DataFrame()

In [4]:
for year in AVAILABLE_YEARS:
    year_data = pd.read_excel(BASE_FILE_PATH.format(year=year), skiprows=4)
    year_data['Ano'] = year

    disasters_data = pd.concat([disasters_data, year_data])


In [5]:
CATEGORIES_TO_REMOVE = ['15110 - Doenças infecciosas virais', '15120 - Doenças infecciosas bacterianas', '15210 - Infestações de animais', '15230 - Outras Infestações', '22410 - Transporte de produtos perigosos rodoviário', '23110 - Incêndios em plantas e distritos industriais, parques e depósitos', '23120 - Incêndios em aglomerados residenciais', '24100 - Colapso de edificações', '25100 - Transporte passageiros e cargas não perigosas rodoviário']
disasters_data = disasters_data.drop(disasters_data[disasters_data['COBRADE'].isin(CATEGORIES_TO_REMOVE)].index)

## 2. Clustering of Regions Regarding Natural Disaster Affects

The first thing we need to do is group the data that we have (from each municipality) into "intro-regions", as shown in the image bellow. 
![](../Docs/Regiões%20Intermediárias.png)

The data of which intro-region delimitation can be found at IBGE's web-site.

In [6]:
REGIONS_DELIMITATION = pd.read_excel('../Data/regioes_geograficas_composicao_por_municipios_2017_20180911.xls')
REGIONS_DELIMITATION = REGIONS_DELIMITATION[['nome_mun', 'nome_rgint']]
REGIONS_DELIMITATION = REGIONS_DELIMITATION[REGIONS_DELIMITATION['nome_rgint'].isin(['Porto Alegre', 'Pelotas', 'Santa Maria', 'Uruguaiana', 'Ijuí', 'Passo Fundo', 'Caxias do Sul', 'Santa Cruz do Sul - Lajeado'])]

In [7]:
#Because of Interface error between IBGE database and S2ID database, one municipality's name does not match correctly and therefore
#needs to be manually corrected
disasters_data['Município'] = disasters_data['Município'].replace(['Sant`Ana do Livramento'],  "Sant'Ana do Livramento")

In [8]:
disasters_data_by_inter_region = pd.merge(left=disasters_data, right=REGIONS_DELIMITATION, left_on='Município', right_on='nome_mun', how='left')

In [9]:
a = disasters_data_by_inter_region.groupby(['nome_rgint', 'Ano'])[['DH_Mortos', 'DH_Feridos', 'DH_Enfermos','DH_Desabrigados', 'DH_Desalojados', 'DH_Desaparecidos', 'DH_Outros Afetados', 'DM_Unidades Habitacionais Danificadas', 'DM_Unidades Habitacionais Destruídas', 'DM_Unidades Habitacionais Valor']].sum()
a.loc['Porto Alegre']

,DH_Mortos,DH_Feridos,DH_Enfermos,DH_Desabrigados,DH_Desalojados,DH_Desaparecidos,DH_Outros Afetados,DM_Unidades Habitacionais Danificadas,DM_Unidades Habitacionais Destruídas,DM_Unidades Habitacionais Valor
Ano,,,,,,,,,,
2013,3,110,809,4590,41015,0,435263,17843,340,7.376325e+07
2014,0,0,0,1477,4152,0,16653,4714,2,2.718652e+06
2015,0,0,0,406,4409,0,11180,4607,1,3.553000e+07
2016,0,331,8,978,15268,0,76866,11918,3,2.502128e+07
2017,0,2,1,38,5371,0,58866,3891,51,3.124366e+07
2018,1,5,1,6,354,0,29548,438,26,7.279949e+05
2019,0,5,0,8,1279,0,40107,1007,2,3.501939e+06
2020,0,0,133,0,16,0,50109,51,4,4.250000e+04
2021,0,1,0,0,68,0,16765,161,1,5.670000e+04


In [15]:
disasters_data_by_inter_region_average = disasters_data_by_inter_region.groupby(['nome_rgint', 'Ano'])[['DH_Mortos', 'DH_Feridos', 'DH_Enfermos','DH_Desabrigados', 'DH_Desalojados', 'DH_Desaparecidos', 'DH_Outros Afetados', 'DM_Unidades Habitacionais Danificadas', 'DM_Unidades Habitacionais Destruídas', 'DM_Unidades Habitacionais Valor']].sum().groupby(['nome_rgint']).mean()
disasters_data_by_inter_region_average

,DH_Mortos,DH_Feridos,DH_Enfermos,DH_Desabrigados,DH_Desalojados,DH_Desaparecidos,DH_Outros Afetados,DM_Unidades Habitacionais Danificadas,DM_Unidades Habitacionais Destruídas,DM_Unidades Habitacionais Valor
nome_rgint,,,,,,,,,,
Caxias do Sul,0.272727,9.000000,0.000000,73.363636,465.545455,0.000000,34657.090909,624.454545,18.272727,1.585272e+07
Ijuí,0.363636,4.545455,0.272727,22.000000,82.181818,0.000000,124601.545455,1449.636364,7.545455,2.469623e+06
Passo Fundo,0.363636,8.818182,6.090909,490.454545,543.727273,0.000000,203136.090909,3309.000000,22.818182,4.829483e+06
Pelotas,0.090909,0.000000,0.090909,96.454545,489.000000,0.000000,159559.454545,3090.000000,3.545455,2.832020e+06
Porto Alegre,0.818182,312.181818,90.090909,1027.727273,9783.909091,0.272727,138664.545455,8785.363636,50.272727,2.736912e+07
Santa Cruz do Sul - Lajeado,0.454545,2.818182,0.909091,459.090909,1962.909091,83.363636,103256.636364,5045.272727,33.545455,7.341957e+07
Santa Maria,0.000000,3.363636,2.454545,72.272727,1377.545455,0.090909,175817.272727,2108.636364,7.727273,4.076717e+06
Uruguaiana,0.000000,5.181818,1.090909,315.363636,2480.272727,4.090909,94724.727273,1024.090909,26.090909,3.663741e+06


### Possibility Functions

Calculate the possibility functions for each criteria. The possibility functions are represented as $[x_1, x_2, x_3, x_4]$

In [27]:
data_min = disasters_data_by_inter_region_average.min()
data_max = disasters_data_by_inter_region_average.max()
data_mean = disasters_data_by_inter_region_average.mean()

lower_measure_possibility_function = {}
lower_measure_possibility_function['x3'] = (data_min + data_mean)/2
lower_measure_possibility_function['x4'] = data_mean

moderate_measure_possibility_function = {}
moderate_measure_possibility_function['x1'] = (data_min + data_mean)/2
moderate_measure_possibility_function['x2'] = data_mean
moderate_measure_possibility_function['x4'] = (data_max + data_mean)/2

upper_measure_possibility_function = {}
upper_measure_possibility_function['x3'] = data_mean
upper_measure_possibility_function['x4'] = (data_max + data_mean)/2

## 3. Clustering of Natural Disaster Season (Year)

In [11]:
disasters_data = disasters_data[['Ano', 'COBRADE', 'DH_Mortos', 'DH_Feridos', 'DH_Enfermos',
                                 'DH_Desabrigados', 'DH_Desalojados', 'DH_Desaparecidos',
                                 'DH_Outros Afetados', 'DM_Unidades Habitacionais Danificadas',
                                 'DM_Unidades Habitacionais Destruídas',
                                 'DM_Unidades Habitacionais Valor',
                                 'DA_Poluição ou contaminação da água',
                                 'DA_Poluição ou contaminação do ar',
                                 'DA_Poluição ou contaminação do solo',
                                 'DA_Diminuição ou exaurimento hídrico'
                                ]]

In [12]:
disasters_data.groupby(['Ano']).sum()

,COBRADE,DH_Mortos,DH_Feridos,DH_Enfermos,DH_Desabrigados,DH_Desalojados,DH_Desaparecidos,DH_Outros Afetados,DM_Unidades Habitacionais Danificadas,DM_Unidades Habitacionais Destruídas,DM_Unidades Habitacionais Valor,DA_Poluição ou contaminação da água,DA_Poluição ou contaminação do ar,DA_Poluição ou contaminação do solo,DA_Diminuição ou exaurimento hídrico
Ano,,,,,,,,,,,,,,,
2013,13215 - Tempestade Local/Convectiva - Vendaval...,5,115,812,12380,46429,1,531830,24734,360,8.715458e+07,0,0,0,0
2014,12200 - Enxurradas13215 - Tempestade Local/Con...,1,71,29,2516,15521,0,264969,16362,229,3.035127e+07,DE 5% A 10%,0,0,0
2015,12100 - Inundações12100 - Inundações13214 - Te...,0,24,0,2908,23701,46,351192,39584,147,9.570269e+07,DE 0% A 5%DE 5% A 10%MAIS DE 20%MAIS DE 20%DE ...,MAIS DE 20%MAIS DE 20%MAIS DE 20%,DE 0% A 5%DE 5% A 10%MAIS DE 20%MAIS DE 20%DE ...,MAIS DE 20%DE 5% A 10%DE 10% A 20%
2016,13215 - Tempestade Local/Convectiva - Vendaval...,2,346,8,1440,18321,916,243990,16636,223,7.186598e+08,DE 0% A 5%DE 5% A 10%DE 10% A 20%DE 10% A 20%D...,DE 0% A 5%,DE 0% A 5%DE 0% A 5%DE 0% A 5%DE 10% A 20%DE 1...,DE 0% A 5%DE 10% A 20%DE 0% A 5%
2017,13213 - Tempestade Local/Convectiva - Granizo1...,5,133,1,638,18171,0,907599,30764,442,7.680782e+07,MAIS DE 20%DE 10% A 20%DE 0% A 5%DE 0% A 5%MAI...,DE 0% A 5%DE 10% A 20%,DE 0% A 5%MAIS DE 20%DE 0% A 5%MAIS DE 20%DE 0...,MAIS DE 20%DE 0% A 5%MAIS DE 20%MAIS DE 20%DE ...
2018,13213 - Tempestade Local/Convectiva - Granizo1...,4,44,4,243,2111,0,353810,9449,72,2.521029e+07,DE 0% A 5%MAIS DE 20%DE 0% A 5%DE 0% A 5%MAIS ...,DE 10% A 20%DE 10% A 20%DE 10% A 20%DE 10% A 20%,DE 5% A 10%MAIS DE 20%DE 0% A 5%DE 5% A 10%MAI...,DE 5% A 10%MAIS DE 20%DE 0% A 5%DE 10% A 20%DE...
2019,14110 - Estiagem14110 - Estiagem14110 - Estiag...,1,6,1,339,3579,0,501589,15539,9,2.827599e+07,DE 0% A 5%DE 5% A 10%MAIS DE 20%DE 5% A 10%DE ...,DE 0% A 5%,DE 0% A 5%DE 5% A 10%DE 0% A 5%DE 0% A 5%DE 10...,DE 5% A 10%DE 0% A 5%MAIS DE 20%DE 5% A 10%MAI...
2020,13215 - Tempestade Local/Convectiva - Vendaval...,0,8,187,17,70,0,1252656,1187,4,1.892718e+06,MAIS DE 20%MAIS DE 20%DE 5% A 10%DE 10% A 20%D...,DE 0% A 5%DE 0% A 5%DE 10% A 20%DE 0% A 5%DE 0...,DE 0% A 5%MAIS DE 20%DE 0% A 5%DE 0% A 5%DE 0%...,MAIS DE 20%MAIS DE 20%MAIS DE 20%MAIS DE 20%MA...
2021,14110 - Estiagem14110 - Estiagem14110 - Estiag...,0,27,0,12,397,0,561831,38537,28,1.563735e+08,MAIS DE 20%DE 0% A 5%DE 0% A 5%,DE 0% A 5%,DE 10% A 20%,DE 5% A 10%DE 5% A 10%MAIS DE 20%MAIS DE 20%MA...
